<a href="https://colab.research.google.com/github/snpushpi/6.804-Computational-Cognitive-Science/blob/master/Normal_Particle_Filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import random
import math
import scipy.stats

In [2]:
state_set = {'A','B','C','D'}
mu_dict = {'A':.2,'B':.4,'C':.6,'D':.8}
sigma = 0.1

In [3]:
def weight_based_sampling(S): #[[state,weight]]
    states=[e[0] for e in S] 
    weights = [e[1] for e in S]
    state =  np.random.choice(states,p=weights) #states [[][]]
    weight = None
    for elt in S:
        if elt[0]==state:
            weight = elt[1]
    return state,weight

In [4]:
def weight_calculate(S):
    weight_dict = {'A':0,'B':0,'C':0,'D':0}
    for state,weight in S:
        weight_dict[state]+=weight
    return weight_dict

In [5]:
class Particle_Filter():

    def __init__(self,particle_num,r):
        self.n = particle_num
        self.S = [[random.choice(tuple(state_set)),1/particle_num] for i in range(particle_num)]
        self.r = r
        self.prediction = None
        self.similarity = None
        self.weight_dict = None

    def update(self,observation, human_observation):
        '''self.S gets updated at each trial.'''
        S_new = [] 
        eta = 0 
        for i in range(self.n):
            state,weight = weight_based_sampling(self.S) #a particle 
            x1 = random.uniform(0,1) 
            if x1<self.r: #change state - (1-e**(-r*k)) =>
                new_set = state_set-{state}
                state= random.choice(tuple(new_set))
            new_weight = scipy.stats.norm(mu_dict[state],sigma).pdf(observation)
            eta+= new_weight
            S_new.append([state,new_weight])
        S_new = [[elt[0],elt[1]/eta] for elt in S_new]
        self.weight_dict = weight_calculate(S_new)
        self.prediction = max(self.weight_dict, key=self.weight_dict.get)
        self.similarity = self.weight_dict[human_observation]
        self.S = S_new

def run(particle_number,observation_list,human_inference_list,r):
    PF = Particle_Filter(particle_number,r)
    model_inference = []
    for i in range(len(observation_list)):
        PF.update(observation_list[i],human_inference_list[i])
        model_inference.append([PF.prediction, PF.similarity])
    return model_inference

In [12]:
def accuracy_and_similarity(human_inference_list, particle_number, observation_list, actual_list, r):
    '''This function returns both the task accuracy and human similarity measure of the normal
    particle filter model'''
    model_inference_list = run(particle_number,observation_list,human_inference_list,r)
    human_similarity_measure = 0
    task_counter = 0
    for i in range(len(observation_list)):
        human_similarity_measure+=model_inference_list[i][1]
        if actual_list[i]==model_inference_list[i][0]:
            task_counter+=1
    return human_similarity_measure/len(observation_list),task_counter/len(observation_list)


In [7]:
def data_generate(alpha,T):
    '''Goal is generate T observations with state changing probability alpha.
    Step 1: Set Z_0 as a random sample from state list.
    Step 2: Repeat the following for T trials -
          i) Z_i = Z_i-1
          ii)Sample x randomly from [0,1]
          iii) If x<alpha replace Z_i with random sample {A,B,C,D}/Z_i-1
          iv)Sample stimulus y_i form a normal distribution with std sigma and mu_zi
    '''
    observation_list = []
    Z = [None]*(T+1)
    Z[0] = random.choice(tuple(state_set))
    for i in range(1,T+1):
        Z[i]=Z[i-1]
        x = random.uniform(0,1)
        if x<alpha:
            new_set = state_set-{Z[i-1]}
            Z[i]= random.choice(tuple(new_set))
        observation_list.append(random.gauss(mu_dict[Z[i]],sigma))        
    return observation_list,Z[1:]


In [8]:
observation_list = [0.2876251534356482,
 0.3093255562034109,
 0.48407616095419964,
 0.4837637156934066,
 0.3429542790801123,
 0.8145469658341187,
 0.5052333561135628,
 0.566893813089657,
 0.5190551990406111,
 0.6756708844580039,
 0.5941568333778058,
 0.6689426081101612,
 0.08007089954529747,
 0.24591174989381062,
 0.13612431798914626,
 0.16855115278807364,
 0.02849659237553981,
 0.2997500771551358,
 0.3786856816409396,
 0.17848722367921957]

In [9]:
actual_list = ['B',
 'B',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'D',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A']

In [13]:
human_inference_list = ['B','B','C','C','C','C','C','D','D','D','D','D','A','A',
 'A','A','A','A','B','B']
particle_number = 100
r = 0.2
particle_filter_num = 10
accuracy_and_similarity(human_inference_list, particle_number, observation_list, actual_list, r)

(0.3940216937507329, 0.6)